In [ ]:
import midi_manipulation
import glob
import numpy as np
import pygame
import os
import midi
import pickle
import pretty_midi

In [ ]:
def midiToNoteStateMatrix(pattern):

    lowerBound = 0
    upperBound = 128
    span = upperBound-lowerBound

    timeleft = [track[0].tick for track in pattern]
    
    posns = [0 for track in pattern]

    statematrix = []
    time = 0

    state = [0 for x in range(span)]
    statematrix.append(state)
    condition = True
    while condition:
        if time % (pattern.resolution / 4) == (pattern.resolution / 8):
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [oldstate[x] for x in range(span)]
            statematrix.append(state)
        for i in range(len(timeleft)): #For each track
            if not condition:
                break
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        pass
                        # print "Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time)
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch] = 0
                        else:
                            state[evt.pitch] = 1
                elif isinstance(evt, midi.TimeSignatureEvent):
                    if evt.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        out =  statematrix
                        condition = False
                        break
                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1

        if all(t is None for t in timeleft):
            break

        time += 1
    statematrix = np.asarray(statematrix).tolist()
    return statematrix

In [ ]:
def noteStateMatrixToMidi(statematrix, name="example", res = 220):
    lowerBound = 0
    upperBound = 128
    span = upperBound-lowerBound

    statematrix = np.array(statematrix)
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    
    #span = upperBound-lowerBound
    tickscale = 55
    
    lastcmdtime = 0
    prevstate = [0 for x in range(span)]
    for time, state in enumerate(np.vstack((statematrix ,prevstate))): 
        offNotes = []
        onNotes = []
        for i in range(span):
            n = state[i]
            p = prevstate[i]
            if p == 1:
                if n == 0:
                    offNotes.append(i)
                elif n == 1:
                    #offNotes.append(i)
                    #onNotes.append(i)
                    pass
            elif n == 1:
                onNotes.append(i)
        for note in offNotes:
            track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note+lowerBound))
            lastcmdtime = time
        for note in onNotes:
            track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=120, pitch=note+lowerBound))
            lastcmdtime = time
            
        prevstate = state
    
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    return pattern
    #midi.write_midifile("{}.mid".format(name), pattern)

In [ ]:
def splitIntoBars(pr1, pr2, pr3):
    ret = []
    ret2 = []
    flag = False
    (r1,c1) = pr1.shape
    (r2,c2) = pr2.shape
    (r3,c3) = pr3.shape
    sizes = [r1, r2, r3]
    rows = min(sizes)
    if r1 != r2 or r1 != r3:
        pr1 = pr1[:rows, :]
        pr2 = pr2[:rows, :]
        pr3 = pr3[:rows, :]
    i = 1
    while rows >= i*16:
        midi.write_midifile("chroma_temp.mid", noteStateMatrixToMidi(pr2[(i)*16-16 : (i)*16,:]))
        midi_data = pretty_midi.PrettyMIDI('chroma_temp.mid')
        try:
            chroma = midi_data.get_chroma(fs=1.0/midi_data.get_end_time())
            ret.append([pr1[(i)*16-16 : (i)*16,:], pr2[(i)*16-16 : (i)*16,:], pr3[(i)*16-16 : (i)*16,:]])
            ret2.append(chroma)
        except ZeroDivisionError:
            flag = True
        i += 1
    return np.asarray(ret), np.asarray(ret2), flag

In [ ]:
poly_bars = []
poly_chroma = []
cnt = 0
errors = 0
ignored = 0
for root, subFolders, files in os.walk('TheoryTab'):
        for f in files:
            if '.mid' in f:
                try:
                    pattern = midi.read_midifile(os.path.join(root, f))
                    cnt += 1
                    print "Processing file ",f,"#", cnt
                    if len(pattern) < 4:
                        print f, 'missing all tracks'
                        errors += 1
                        continue
                    track1 = midi.Pattern(resolution = pattern.resolution)
                    track1.append(pattern[0])
                    track1.append(pattern[1])
                    pr1 = np.asarray(midiToNoteStateMatrix(track1))
                    track2 = midi.Pattern(resolution = pattern.resolution)
                    track2.append(pattern[0])
                    track2.append(pattern[2])
                    pr2 = np.asarray(midiToNoteStateMatrix(track2))
                    track3 = midi.Pattern(resolution = pattern.resolution)
                    track3.append(pattern[0])
                    track3.append(pattern[3])
                    pr3 = np.asarray(midiToNoteStateMatrix(track3))
                    res,res2, flag = splitIntoBars(pr1,pr2,pr3)
                    if flag == True:
                        ignored += 1
                    poly_bars.extend(res)
                    poly_chroma.extend(res2)
                except TypeError:
                    print f, 'raised error'
                    errors += 1

print errors,ignored, cnt

In [ ]:
print np.array(poly_bars).shape
print np.array(poly_chroma).reshape((len(poly_chroma),12)).shape
np.save('poly_bars',np.array(poly_bars))
np.save('poly_chroma',np.array(poly_chroma).reshape((len(poly_chroma),12)))

In [ ]:
pygame.init()
def play(midiFile):
    pygame.mixer.music.load(midiFile)
    pygame.mixer.music.play()

In [ ]:
#playing the resultant midi produced by the generator.
result = np.load('result_poly.npy')
t1 = noteStateMatrixToMidi(result[0])
t2 = noteStateMatrixToMidi(result[1])
t3 = noteStateMatrixToMidi(result[2])
song =  midi.Pattern()
song.append(t1[0])
song.append(t2[0])
song.append(t3[0])
midi.write_midifile("result_poly_tune.mid", song)
play('result_poly_tune.mid')